In [464]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

from sklearn.model_selection import train_test_split, cross_val_score

import lightgbm as lgb

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_log_error

from sklearn.ensemble import VotingRegressor

from sklearn import linear_model

In [465]:
train  = pd.read_csv('../MachineHackE-Commerce/Train.csv')
test = pd.read_csv('../MachineHackE-Commerce/Test.csv')
sub = pd.read_excel('../MachineHackE-Commerce/Sample_Submission.xlsx')


In [466]:
test.tail()

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date
1046,P-9758,B-2543,sports fitness,outdoor adventure,camping hiking,2.0,28/2/2014
1047,P-11898,B-3197,jewellery,necklaces chains,necklaces,4.7,1/12/2019
1048,P-904,B-133,automotive,accessories spare parts,car interior exterior,3.5,1/12/2014
1049,P-1714,B-358,kitchen dining,lighting,bulbs,2.4,23/6/2013
1050,P-620,B-133,automotive,accessories spare parts,car interior exterior,3.1,1/12/2012


In [467]:
z =train[ train['Item_Category']=='automotive' ]

In [468]:
z[z['Subcategory_2']=='car interior exterior']

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price
9,P-360,B-88,automotive,accessories spare parts,car interior exterior,2.3,10/5/2018,199.0
56,P-10422,B-2741,automotive,accessories spare parts,car interior exterior,3.4,21/4/2014,350.0
69,P-11595,B-3106,automotive,accessories spare parts,car interior exterior,3.5,24/5/2014,592.0
73,P-1004,B-133,automotive,accessories spare parts,car interior exterior,4.2,1/12/2017,1374.0
108,P-612,B-133,automotive,accessories spare parts,car interior exterior,3.9,1/12/2013,1348.0
...,...,...,...,...,...,...,...,...
2342,P-715,B-133,automotive,accessories spare parts,car interior exterior,2.6,1/12/2012,1594.0
2344,P-920,B-133,automotive,accessories spare parts,car interior exterior,4.3,1/12/2019,919.0
2347,P-975,B-133,automotive,accessories spare parts,car interior exterior,4.5,1/12/2019,1380.0
2436,P-829,B-133,automotive,accessories spare parts,car interior exterior,2.0,1/12/2017,1348.0


In [469]:
test

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date
0,P-11284,B-2984,computers,network components,routers,4.3,1/12/2018
1,P-6580,B-1732,jewellery,bangles bracelets armlets,bracelets,3.0,20/12/2012
2,P-5843,B-3078,clothing,women s clothing,western wear,1.5,1/12/2014
3,P-5334,B-1421,jewellery,necklaces chains,necklaces,3.9,1/12/2019
4,P-5586,B-3078,clothing,women s clothing,western wear,1.4,1/12/2017
...,...,...,...,...,...,...,...
1046,P-9758,B-2543,sports fitness,outdoor adventure,camping hiking,2.0,28/2/2014
1047,P-11898,B-3197,jewellery,necklaces chains,necklaces,4.7,1/12/2019
1048,P-904,B-133,automotive,accessories spare parts,car interior exterior,3.5,1/12/2014
1049,P-1714,B-358,kitchen dining,lighting,bulbs,2.4,23/6/2013


In [470]:

train_enc = pd.read_csv('../MachineHackE-Commerce/train_enc.csv')

In [471]:
test_enc = pd.read_csv('../MachineHackE-Commerce/test_enc.csv')

In [472]:
train_enc.columns

Index(['Product', 'Product_Brand', 'Item_Category', 'Subcategory_1',
       'Subcategory_2', 'Item_Rating', 'Selling_Price', 'Is_Holiday', 'Year',
       'Month', 'Week', 'Day', 'Dayofweek', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start'],
      dtype='object')

In [473]:
train_enc1 = train_enc.drop(['Product', 'Product_Brand', 'Item_Category', 'Subcategory_1',
       'Subcategory_2','Selling_Price'],axis=1)*1
test_enc1 = test_enc.drop(['Product', 'Product_Brand', 'Item_Category', 'Subcategory_1',
       'Subcategory_2'],axis=1)*1

In [474]:
# def f(s):
#     return (s-s.mean())/s.std()
def f(s):
    return s/s.max()

def fi(s):
    return s/10


In [475]:
temptrain = f_y(train_enc1['Year'])
temptest = f_y(test_enc1['Year'])

In [476]:
train_enc1['Year'] = temptrain
test_enc1['Year'] = temptest

In [477]:
# def f(s):
#     return s/s.max()

In [478]:
train_enc1.columns

Index(['Item_Rating', 'Is_Holiday', 'Year', 'Month', 'Week', 'Day',
       'Dayofweek', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start'],
      dtype='object')

In [479]:
train_enc1['Month'] = train_enc1[['Month']].apply(f)
test_enc1['Month'] = test_enc1[['Month']].apply(f)


train_enc1['Week'] = train_enc1[['Week']].apply(f)
test_enc1['Week'] = test_enc1[['Week']].apply(f)

train_enc1['Day'] = train_enc1[['Day']].apply(f)
test_enc1['Day'] = test_enc1[['Day']].apply(f)

train_enc1['Dayofweek'] = train_enc1[['Dayofweek']].apply(f)
test_enc1['Dayofweek'] = test_enc1[['Dayofweek']].apply(f)

train_enc1['Item_Rating'] = train_enc1[['Item_Rating']].apply(fi)
test_enc1['Item_Rating'] = test_enc1[['Item_Rating']].apply(fi)


In [480]:
train_enc1.head()

,Item_Rating,Is_Holiday,Year,Month,Week,Day,Dayofweek,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start
0,0.43,0,0.998515,0.166667,0.094340,0.096774,0.666667,0,0,0,0
1,0.31,0,0.997525,0.583333,0.509434,0.032258,0.333333,0,1,0,1
2,0.35,1,0.999505,0.083333,0.037736,0.387097,0.833333,0,0,0,0
3,0.40,0,0.997030,1.000000,0.943396,0.387097,0.666667,0,0,0,0
4,0.43,0,0.996535,1.000000,0.943396,0.387097,0.500000,0,0,0,0


In [481]:
encoded_train = train_enc1.values

encoded_test = test_enc1.values


In [482]:
NUM_BRANDS = 790
NUM_CATEGORIES = 1001
NAME_MIN_DF = 10
MAX_FEATURES_ITEM_DESCRIPTION = 3

In [483]:
y = np.log1p(train["Selling_Price"])
      

In [484]:

train['text'] = train.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)
test['text'] = test.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)



In [485]:
cv = CountVectorizer(min_df=NAME_MIN_DF)
#X_Item_Category = cv.fit_transform(train['Item_Category'])


X_Item_Category_train = cv.fit_transform(train['Item_Category'])

X_Item_Category_test = cv.transform(test['Item_Category'])

    

In [486]:
cv = CountVectorizer()
#X_category1 = cv.fit_transform(dataset['Subcategory_1'])

X_category1_train = cv.fit_transform(train['Subcategory_1'])
X_category1_test = cv.transform(test['Subcategory_1'])
    

In [487]:
cv2 = CountVectorizer()

#X_category2 = cv.fit_transform(dataset['Subcategory_2'])
    
X_category2_train = cv.fit_transform(train['Subcategory_2'])
X_category2_test = cv.transform(test['Subcategory_2'])
    

In [488]:
tv = TfidfVectorizer(max_features=MAX_FEATURES_ITEM_DESCRIPTION,
                         ngram_range=(1, 3),
                         stop_words='english')
X_description_train = tv.fit_transform(train['text'])

X_description_test = tv.transform(test['text'])
    

In [489]:
lb = LabelBinarizer(sparse_output=True)
X_brand_train = lb.fit_transform(train['Product_Brand'])
X_brand_test = lb.transform(test['Product_Brand'])
    

In [490]:
train['Product'] = train['Product'].str.replace(r'\D', '')

train['Product'] = pd.to_numeric(train['Product'], errors='coerce')


test['Product'] = test['Product'].str.replace(r'\D', '')

test['Product'] = pd.to_numeric(test['Product'], errors='coerce')


In [491]:
X_product_train = f(train['Product'])

X_product_test = f(test['Product'])

In [492]:
X_prod_train=np.array(X_product_train).reshape(-1,1)

X_prod_test=np.array(X_product_test).reshape(-1,1)

In [493]:
# X_description.shape
# X_category1.shape
# X_category2.shape
# X_Item_Category.shape
# X_prod.shape

In [494]:
tst = hstack((X_prod_test,X_brand_test,X_Item_Category_test ,X_category1_test,X_category2_test,
              X_description_test,encoded_test)).tocsr()
 

In [495]:
trn = hstack((X_prod_train,X_brand_train,X_Item_Category_train ,X_category1_train,X_category2_train,
              X_description_train,encoded_train)).tocsr()
 

In [496]:
# def rmsle_cv(model,X,y):
#     n_folds = 5
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X)
#     rmsle= np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_log_error", cv = kf))
#     print(rmsle.mean())
#     return(rmsle)

In [497]:
# def rmsle_cv_ensemble(model,X,y):
#     n_folds = 5
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X)
    
#     y_pred = cross_val_predict(model, X, np.log1p(y), cv=kf)
#     if (y_pred<0).sum() :
#             print("negtive {0} value predicted".format( (y_pred<0).sum())) 
          
        
#     y_pred_inverse = np.expm1(y_pred)
    
    
#     score = np.sqrt(mean_squared_log_error(y, y_pred))
    
#     return(score)

In [498]:
def rmsle(y, y_pred):
        return np.sqrt(mean_squared_log_error(y, y_pred))

In [499]:
# d_train = lgb.Dataset(X, label= y)
# params = {
#         'learning_rate': 0.45,
#         'application': 'regression',
#         'max_depth': 10,
#         'num_leaves': 1000,
#         'verbosity': -1,
#         'metric': 'RMSLE'}

# model_lgb = lgb.train(params, train_set=d_train, num_boost_round=3200, verbose_eval=100) 

# model_lgb_train_pred = np.expm1(model_lgb.predict(X))

# model_lgb_pred = np.expm1(model_lgb.predict(X_test))



In [500]:
# model_lgb_pred

In [501]:
# print(rmsle(np.expm1(y), model_lgb_train_pred))

In [502]:
model_ridge =Ridge(solver="sag", fit_intercept=True, random_state=2)

model_ridge.fit(trn, y)

model_ridge_train_pred = np.expm1(model_ridge.predict(trn))

model_ridge_pred = np.expm1(model_ridge.predict(tst))

print(rmsle(np.expm1(y), model_ridge_train_pred))

0.380191533363242


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:558: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  '"sag" solver requires many iterations to fit '


In [503]:
model_ridge_pred

array([2040.81031436,  528.49546039,  638.67487836, ..., 1350.89815909,
        384.17308385, 1464.20986019])

In [340]:
model_ridge_pred

array([2041.86143186,  529.2757357 ,  641.84751517, ..., 1351.3416202 ,
        386.22469122, 1471.93833494])

In [452]:
from sklearn.model_selection import GridSearchCV

In [453]:
# from sklearn.model_selection import RandomizedSearchCV
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 50, stop = 500, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(11, 33, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

In [454]:
# rf = RandomForestRegressor()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# # Fit the random search model
# rf_random.fit(X1, y)

In [455]:
# rf_random.best_params_

In [504]:
model_rf = RandomForestRegressor(n_estimators= 100,
 min_samples_split= 2,
 min_samples_leaf =1,
 max_features= 'sqrt',
 max_depth= 23,
 bootstrap= False,random_state=2)

model_rf.fit(trn, y)
model_rf_train_pred = np.expm1(model_rf.predict(trn))

model_rf_pred = np.expm1(model_rf.predict(tst))
print(rmsle(np.expm1(y), model_rf_train_pred))

0.44800429158807514


In [505]:
model_rf_pred

array([2071.66935783,  543.33885803,  709.77240736, ..., 1024.90906521,
        581.97356273, 1386.36226133])

In [457]:
model_rf_pred

array([2071.66935783,  543.33885803,  709.77240736, ..., 1024.90906521,
        581.97356273, 1386.36226133])

In [347]:
model_rf_pred

array([1827.32949472,  540.55259732,  654.56397604, ...,  994.03595302,
        586.55104105, 1312.64303095])

In [506]:

print('RMSLE score on train data:')
print(rmsle(np.expm1(y), model_rf_train_pred*0.24 + model_ridge_train_pred*0.76))

RMSLE score on train data:
0.3775461841664115


In [507]:
ensemble =  model_rf_pred*0.24 + model_ridge_pred*0.76 

In [508]:
ensemble

array([2048.21648479,  532.05787582,  655.73828532, ..., 1272.66077655,
        431.64519878, 1445.52643646])

In [509]:
ensemble

array([2048.21648479,  532.05787582,  655.73828532, ..., 1272.66077655,
        431.64519878, 1445.52643646])

In [510]:
sub['Selling_Price'] = ensemble

sub.to_excel("submissionfinal.xlsx", index=False)
